In [89]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By

In [90]:
#-------------------Extract-------------------------
urls = ["https://forecast.weather.gov/MapClick.php?lat=57.0826&lon=-135.2692#.Y-vs_9LMJkg",
        'https://forecast.weather.gov/MapClick.php?lat=45.5118&lon=-122.6756#.Y-vtHNLMJkg',
        'https://forecast.weather.gov/MapClick.php?lat=40.7143&lon=-74.006#.Y-vtQtLMJkg'
        ]
        
source_df = pd.DataFrame()

for url in urls:
    r = requests.get(url)
    soup = BeautifulSoup(r.content,"html.parser")

    #various containers
    item1 = soup.find_all(id='current_conditions-summary')
    item2 = soup.find_all(id='current_conditions_detail')

    #raw data
    temp_f = [item.find(class_="myforecast-current-lrg").get_text() for item in item1]
    temp_c = [item.find(class_="myforecast-current-sm").get_text() for item in item1]

    #df of temperatures
    df_temperature = pd.DataFrame({"Temperature_F" : temp_f, "Temperature_C": temp_c})

    #df_2 is a df of current conditions in detail
    table = soup.find_all('table')
    df_2 = pd.read_html(str(table))[0]
    df_2 = df_2.pivot(columns=0, values=1).ffill().dropna().reset_index().drop(columns=['index'])

    #merge both dataframes
    temp_df=pd.concat([df_temperature,df_2],axis=1)

    #scrape lattitude, longitude, and elevation 
    lat_lon_elev = soup.find('span', {'class': 'smallTxt'}).text.strip()
    lat, lon, elev = re.findall(r'[-+]?\d*\.\d+|\d+', lat_lon_elev)

    #scrape name
    location = soup.find('h2', {'class': 'panel-title'}).text.strip()

    #add location, lat, long, and elev to source_df
    temp_df['elevation_ft'] = elev
    temp_df['latitude'] = lat
    temp_df['longitude'] = lon
    temp_df['location'] = location

    source_df = pd.concat([temp_df, source_df], ignore_index=True, sort=False)

display(source_df)



,Temperature_F,Temperature_C,Barometer,Dewpoint,Humidity,Last update,Visibility,Wind Chill,Wind Speed,elevation_ft,latitude,longitude,location
0,54°F,12°C,30.10 in (1018.3 mb),18°F (-8°C),24%,14 Feb 3:51 pm EST,10.00 mi,51°F (11°C),W 9 mph,154,40.78,73.97,"New York City, Central Park (KNYC)"
1,41°F,5°C,30.18 in (1022.01 mb),32°F (0°C),70%,14 Feb 01:20 PM PST,10.00 mi,37°F (3°C),NW 6 MPH,20,45.59578,122.60917,"Portland, Portland International Airport (KPDX)"
2,36°F,2°C,30.11 in (1019.5 mb),25°F (-4°C),64%,14 Feb 11:53 am AKST,10.00 mi,27°F (-3°C),E 14 mph,13,57.05,135.36,Sitka - Sitka Airport (PASI)


In [94]:
URLS = ['https://www.wunderground.com/history/daily/us/ny/new-york-city/KLGA',
        "https://www.wunderground.com/history/daily/us/or/portland",
        'https://www.wunderground.com/history/daily/us/ak/sitka/PASI'
        ]

precip_df = pd.DataFrame()

for URL in URLS:

    driver = webdriver.Chrome(r"C:/home/reed/.cache/selenium/chromedriver/linux64/109.0.5414.74")
    driver.get(URL)
    sleep(5)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    try:
        driver.find_element(By.XPATH,'/html/body/div[6]/div[3]/div/div/div/a').click()
    except:
        precip = driver.find_element(By.XPATH,"/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[2]/tr/td[1]").text

    df = pd.DataFrame([precip], columns=['precip'])
    precip_df = pd.concat([precip_df, df], ignore_index=True, sort=False)


    driver.quit()

source_df =pd.concat([source_df,precip_df],axis=1)
display(source_df)

/home/reed/Desktop/epicodus/team-week3/reed-work/venvteamweek/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  # Remove the CWD from sys.path while we load stuff.


,precip
0,0.00
1,0.14
2,0.11


In [95]:
source_df =pd.concat([source_df,precip_df],axis=1)
source_df

,Temperature_F,Temperature_C,Barometer,Dewpoint,Humidity,Last update,Visibility,Wind Chill,Wind Speed,elevation_ft,latitude,longitude,location,precip
0,54°F,12°C,30.10 in (1018.3 mb),18°F (-8°C),24%,14 Feb 3:51 pm EST,10.00 mi,51°F (11°C),W 9 mph,154,40.78,73.97,"New York City, Central Park (KNYC)",0.00
1,41°F,5°C,30.18 in (1022.01 mb),32°F (0°C),70%,14 Feb 01:20 PM PST,10.00 mi,37°F (3°C),NW 6 MPH,20,45.59578,122.60917,"Portland, Portland International Airport (KPDX)",0.14
2,36°F,2°C,30.11 in (1019.5 mb),25°F (-4°C),64%,14 Feb 11:53 am AKST,10.00 mi,27°F (-3°C),E 14 mph,13,57.05,135.36,Sitka - Sitka Airport (PASI),0.11


In [ ]:
#-------------Transformations------------------------


In [ ]:
import requests
from bs4 import BeautifulSoup
import re

with open('cities.json') as f:
    cities = json.load(f)

for city in cities:
    response = requests.get(city['NWS_URL'])
    soup = BeautifulSoup(response.content, 'html.parser')

    location = soup.find('h2', {'class': 'panel-title'})
    lat_lon_elev = soup.find('span', {'class': 'smallTxt'}).text.strip()
    lat, lon, elev = re.findall(r'[-+]?\d*\.\d+|\d+', lat_lon_elev)
    temperature = soup.find('p', {'class': 'myforecast-current-lrg'})
    humidity_elem = soup.find('td', text='Humidity')
    humidity = humidity_elem.find_next('td').text.strip() if humidity_elem else ''
    wind_speed_elem = soup.find('td', text='Wind Speed')
    wind_speed = wind_speed_elem.find_next('td').text.strip() if wind_speed_elem else ''
    barometer_elem = soup.find('td', text='Barometer')
    barometer = barometer_elem.find_next('td').text.strip() if barometer_elem else ''
    dewpoint_elem = soup.find('td', text='Dewpoint')
    dewpoint = dewpoint_elem.find_next('td').text.strip() if dewpoint_elem else ''
    visibility_elem = soup.find('td', text='Visibility')
    visibility = visibility_elem.find_next('td').text.strip() if visibility_elem else ''
    wind_chill_elem = soup.find('td', text='Wind Chill')
    wind_chill = wind_chill_elem.find_next('td').text.strip() if wind_chill_elem else ''
    last_update_elem = soup.find('td', text='Last update')
    last_update = last_update_elem.find_next('td').text.strip() if last_update_elem else ''

    print({
        'Name': city['Name'],
        'Lat': lat + '°N',
        'Lon': lon + '°W',
        'Elev': elev + 'ft.',
        'Temperature': temperature.text if temperature else '',
        'Humidity': humidity,
        'Wind Speed': wind_speed,
        'Barometer': barometer,
        'Dewpoint': dewpoint,
        'Visibility': visibility,
        'Wind Chill': wind_chill,
        'Last update': last_update
    })